# Taller de búsqueda de hiperparámetros

En este taller vamos a explorar la búsqueda de hiperparámetros de manera automática.

In [1]:
!pip install numpy scikit-learn pandas matplotlib

In [2]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
X, y = load_breast_cancer(return_X_y=True)

In [4]:
test_size=0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

### **Ejemplo 1**
Encuentra los mejores hiperparámetros usando `GridSearchCV`

In [5]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_grid = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

Primero exploremos un poco el dataset

In [6]:
n_samples = X.shape[0]
n_features = X.shape[1]
n_benign = np.sum(y == 1)
n_malignant = np.sum(y == 0)

perc_benign = n_benign / n_samples * 100
perc_malignant = n_malignant / n_samples * 100

print("Resumen del conjunto de datos de cancer de mama:")
print(f"Total de muestras: {n_samples}")
print(f"Número de características: {n_features}")
print("\nClases objetivo:")
print(f"0 = Maligno: {n_malignant} muestras ({perc_malignant:.1f}%)")
print(f"1 = Benigno: {n_benign} muestras ({perc_benign:.1f}%)")

Resumen del conjunto de datos de cancer de mama:
Total de muestras: 569
Número de características: 30

Clases objetivo:
0 = Maligno: 212 muestras (37.3%)
1 = Benigno: 357 muestras (62.7%)


Usamos scoring="f1" en Grid Search porque el dataset está ligeramente desbalanceado y nos interesa que el modelo tenga buen rendimiento tanto en casos benignos como malignos. La métrica F1 permite equilibrar precisión y recall, lo cual es clave en contextos médicos donde un falso negativo puede ser más grave que un falso positivo.

In [7]:
# Utiliza GridSearchCV
gs = GridSearchCV(clf, param_grid, cv=5, scoring='f1', verbose=1)

t0 = time.time()
gs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Tiempo de búsqueda: 0.599s


podemos ver que el atributo `cv_results_` nos entrega los resultados de toda la búsqueda.

In [8]:
dir(gs)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_refit_for_multimetric',
 '_check_scorers_accept_sample_weight',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_html_repr',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [9]:
resultados = pd.DataFrame(gs.cv_results_)
display(resultados)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000496,0.000993,0.003524,0.006345,gini,3,1,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.932039,0.942308,0.952381,0.934579,0.936937,0.939649,0.007213,21
1,0.000649,0.001299,0.002586,0.004708,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.923077,0.942308,0.952381,0.943396,0.936937,0.939620,0.009646,22
2,0.004946,0.004956,0.001028,0.000037,gini,3,3,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.921569,0.962264,0.961538,0.934579,0.945455,0.945081,0.015680,13
3,0.002343,0.001989,0.000397,0.000486,gini,6,1,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",0.914286,0.943396,0.943396,0.943396,0.954128,0.939721,0.013379,20
4,0.000425,0.000850,0.006259,0.007668,gini,6,2,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",0.912621,0.943396,0.962264,0.934579,0.935780,0.937728,0.015989,23
5,0.000000,0.000000,0.000000,0.000000,gini,6,3,"{'criterion': 'gini', 'max_depth': 6, 'min_sam...",0.942308,0.971963,0.961538,0.934579,0.954128,0.952903,0.013322,4
6,0.006431,0.007876,0.000000,0.000000,gini,12,1,"{'criterion': 'gini', 'max_depth': 12, 'min_sa...",0.903846,0.962264,0.962264,0.943396,0.954128,0.945180,0.021797,12
7,0.006362,0.007792,0.000000,0.000000,gini,12,2,"{'criterion': 'gini', 'max_depth': 12, 'min_sa...",0.910891,0.962264,0.941176,0.942308,0.944444,0.940217,0.016548,19
8,0.001101,0.002202,0.003428,0.006363,gini,12,3,"{'criterion': 'gini', 'max_depth': 12, 'min_sa...",0.942308,0.932039,0.961538,0.934579,0.954128,0.944919,0.011319,14
9,0.003132,0.006264,0.000000,0.000000,gini,18,1,"{'criterion': 'gini', 'max_depth': 18, 'min_sa...",0.901961,0.962264,0.971963,0.952381,0.954128,0.948539,0.024301,10


Lo más importante es extraer los hiperparámetros del modelo que mejor error en de validación sacaron

In [10]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 12, 'min_samples_leaf': 2}

También es posible el mejor resultado en la métrica usada

In [11]:
gs.best_score_

0.9601325319771922

Finalmente, es posible extraer directamente un estimador que que ha sido creado con los mejores hiperparámetros.

In [12]:
gs.best_estimator_

,criterion,'entropy'
,splitter,'best'
,max_depth,12
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


Evaluar el mejor modelo en el conjunto de prueba

In [13]:
mejor_modelo = gs.best_estimator_
y_pred = mejor_modelo.predict(X_test)
print("Accuracy en test:", accuracy_score(y_test, y_pred))
print("F1 Score en test:", f1_score(y_test, y_pred))


Accuracy en test: 0.9230769230769231
F1 Score en test: 0.9378531073446328


### **Ejemplo 2** 
La clase `RandomizedSearchCV` se puede usar casi de la misma manera, solo que esta vez se debe escoger un número de combinaciones a evaluar; las cuales se escogeran de manera aleatoria.

In [14]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_dist = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza RandomizedSearchCV
rs = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="accuracy",
    random_state=42,
    verbose=1
)
t0 = time.time()
rs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Tiempo de búsqueda: 0.214s


In [15]:
rs.best_score_

0.9460465116279071

In [16]:
print("Mejores parámetros encontrados:", rs.best_params_)

Mejores parámetros encontrados: {'min_samples_leaf': 1, 'max_depth': 12, 'criterion': 'entropy'}


Evaluar el modelo encontrado por RandomizedSearchCV

In [17]:
mejor_modelo_rs = rs.best_estimator_
y_pred_rs = mejor_modelo_rs.predict(X_test)

print("Accuracy en test:", accuracy_score(y_test, y_pred_rs))
print("F1 Score en test:", f1_score(y_test, y_pred_rs))

Accuracy en test: 0.9440559440559441
F1 Score en test: 0.9555555555555556


Podemos ver que aunque se demoró mucho menos, el resultado no es tan bueno.

### Búsqueda de hiperparámetros usando Optuna

Optuna es una biblioteca moderna y eficiente para la optimización automática de hiperparámetros. A diferencia de GridSearchCV o RandomizedSearchCV, utiliza técnicas de búsqueda más inteligentes, como optimización bayesiana, para encontrar buenos resultados con menos combinaciones.

In [30]:
!pip install --upgrade nbformat ipython plotly optuna

In [31]:
import optuna
import optuna.visualization as vis
import plotly.io as pio
from sklearn.model_selection import cross_val_score

Definimos la función objetivo que Optuna utilizará para evaluar diferentes combinaciones de hiperparámetros. Cada combinación será evaluada usando validación cruzada y la métrica F1.

In [39]:
# Definimos la función objetivo
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    
    clf = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42
    )

    score = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
    return score.mean()

Ahora creamos un estudio y permitimos que Optuna explore 30 combinaciones diferentes de hiperparámetros. Medimos también el tiempo que tarda en ejecutarse.

In [42]:
study = optuna.create_study(direction="maximize")
t0 = time.time()
study.optimize(objective, n_trials=30)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))


[I 2025-07-12 22:11:37,362] A new study created in memory with name: no-name-8770198c-a48a-4ac2-9eb8-0acc4c3675b9
[I 2025-07-12 22:11:37,391] Trial 0 finished with value: 0.941340629274966 and parameters: {'max_depth': 8, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 0 with value: 0.941340629274966.
[I 2025-07-12 22:11:37,425] Trial 1 finished with value: 0.941313269493844 and parameters: {'max_depth': 7, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 0 with value: 0.941340629274966.
[I 2025-07-12 22:11:37,439] Trial 2 finished with value: 0.9412585499316005 and parameters: {'max_depth': 5, 'min_samples_leaf': 5, 'criterion': 'entropy'}. Best is trial 0 with value: 0.941340629274966.
[I 2025-07-12 22:11:37,477] Trial 3 finished with value: 0.934281805745554 and parameters: {'max_depth': 9, 'min_samples_leaf': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.941340629274966.
[I 2025-07-12 22:11:37,505] Trial 4 finished with value: 0.9225991792065663 

Tiempo de búsqueda: 1.004s


Podemos consultar los mejores hiperparámetros encontrados y el mejor resultado de validación cruzada:

In [43]:
best_params = study.best_params
print("Mejores hiperparámetros encontrados por Optuna:")
print(best_params)
print("Mejor score de validación (f1):", study.best_value)

Mejores hiperparámetros encontrados por Optuna:
{'max_depth': 7, 'min_samples_leaf': 3, 'criterion': 'entropy'}
Mejor score de validación (f1): 0.94593707250342


Finalmente, evaluamos el modelo con los mejores parámetros sobre el conjunto de prueba.

In [44]:
final_model = DecisionTreeClassifier(**best_params, random_state=42)
final_model.fit(X_train, y_train)
y_pred_optuna = final_model.predict(X_test)

print("Accuracy en test (Optuna):", accuracy_score(y_test, y_pred_optuna))
print("F1 Score en test (Optuna):", f1_score(y_test, y_pred_optuna))

Accuracy en test (Optuna): 0.9370629370629371
F1 Score en test (Optuna): 0.9491525423728814


In [45]:
print("Modelo final:")
print(final_model)

Modelo final:
DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_leaf=3,
                       random_state=42)


También es posible visualizar los resultados del proceso de optimización. Esto nos ayuda a entender cómo evolucionó el score a lo largo de los intentos, y qué hiperparámetros fueron más importantes.

In [46]:
# Establecer el renderizador a 'browser'
pio.renderers.default = 'browser'

# Ahora sí mostrar el gráfico
import optuna.visualization as vis
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()